# ARMA(p,q) 同定
ARMA(p,q)システム，(p,q)を様々に変えてデータを生成して，パラメータ推定の方法と評価の仕方を述べる。<br>

#### モデルの説明
ARMAはARIIMAに含まれているのでARIMAモデルの説明を使用。参考までにARモデルのサイトも記す。
- AR(p) model  https://www.statsmodels.org/devel/examples/notebooks/generated/autoregressions.html
- ARIMA model https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMA.html

#### データ生成は次の関数を使用
- statsmodels.tsa.arima_process.arma_generate_sample https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_process.arma_generate_sample.html

#### 各種関数（メソッド）の説明
- statsmodels.tsa.arima.model.ARIMA.fit https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMA.fit.html
- statsmodels.tsa.arima.model.ARIMAResults https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMAResults.html#statsmodels.tsa.arima.model.ARIMAResults
- statsmodels.tsa.arima.model.ARIMAResults.plot_diagnostics https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMAResults.plot_diagnostics.html
- statsmodels.tsa.arima_model.ARIMAResults.plot_predict https://www.statsmodels.org/v0.11.1/generated/statsmodels.tsa.arima_model.ARIMAResults.plot_predict.html
- trend="n","c","t","ct"の説明　https://www.statsmodels.org/dev/generated/statsmodels.tsa.deterministic.TimeTrend.from_string.html

#### 例題：
- Artificial data https://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_1.html
- Sunspots data https://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_0.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy as sp

import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.arima_process import arma_impulse_response
from statsmodels.graphics.tsaplots import plot_predict

import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline

#### データ生成 arma_generate_sample<br>
引数の説明： 
- maの係数は$b_0$項を含める。
- distrvs: ARMAへの入力でディフォルトは標準正規分布(0, 1) as np.random.randn
- burnin: 初期期間を何点かで定め，この期間後にデータを出力する。過渡現象を観測しないため。

#### 注意
書籍では生成するデータ数をNだけとしたが，本スクリプトでは下記のNall（N＋テストデータ数）とした。

In [ ]:
np.random.seed(123)
N = 2000 #データ数
N1 = 100 #テストデータ数
Nall = N+N1
ar, ma = [1, -1.5, 0.7], [1.0, 0.6]
 
# ARMAモデルへの入力を関数として与える。
input = lambda size: np.random.normal(loc=0.0, scale=1.0, size=size) 
#input = lambda n: np.random.standard_t(3, size=n)  # 他の例：t 分布

# ARMAモデルの入力 = scale*input
yall = arma_generate_sample(ar=ar, ma=ma, nsample=Nall, scale=1, distrvs=input, burnin=500)
y = yall[:N]
ytest = yall[N:]
print(len(y), len(ytest))

#### 極の大きさの確認（安定か否かの確認）

In [ ]:
poles = np.roots(ar)
print('poles:', poles,'  abs(poles):', np.abs(poles))# 単位円内にあるかの確認

#### 自己相関関数
numpy.correlate　https://numpy.org/doc/stable/reference/generated/numpy.correlate.html

In [ ]:
autocorr = np.correlate(y, y, mode='full')
autocorr = autocorr/autocorr.max() # normalization

#### 観測データyと自己相関関数のプロット
- 観測データは一部のみをプロット
- 自己相関関数のプロットで，nn = int(len(autocorr)/2)としている理由は，自己相関関数が原点で左右対称という性質を持つため，右半分だけをプロットするため。
- lagの数は，適当に定めて試されたい

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,2))

ax[0].plot(y[0:500], c='k')
ax[0].set_xlabel('k')
ax[0].set_ylabel('y')
ax[0].grid()

lag=40
nn = int(len(autocorr)/2)  # 自己相関関数の配列の中心が横軸のゼロに相当することを利用，ここを出発点とする。
ax[1].plot(autocorr[nn:nn+lag], marker='x', c='k')
ax[1].set_xlabel('lag')
ax[1].set_ylabel('acf')
ax[1].grid()

#plt.savefig('fig_ARMAident_y_acf.pdf', bbox_inches='tight')
plt.show()

真の次数は未知として，幾つかの推定次数を用い，比較する<br>
もし，"FutureWarning"というWarningは無視して構わない（2018年現在）

In [ ]:
arma20 = ARIMA(y, order=(2,0,0), trend='n').fit()

In [ ]:
arma21 = ARIMA(y, order=(2,0,1), trend='n').fit()

In [ ]:
arma32 = ARIMA(y, order=(3,0,2), trend='n').fit()

In [ ]:
arma43 = ARIMA(y, order=(4,0,3), trend='n').fit()
print(arma43.mle_retvals) # retvals is short for return values

#### 上記のWarningの説明
"Maximum Likelihood optimization failed to converge"は収束計算に失敗したことを述べており，このため，この結果は用いない。失敗の説明は<br>
statsmodels.tsa.arima.model.ARIMA.fit の次にある。
- https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMA.fit.html<br>
より，パラメータ推定の計算にはgls(Generalized least squares) を使用
- https://en.wikipedia.org/wiki/Generalized_least_squares
を使用。GLSの特徴は下記を参照<br>
- 英語Wiki https://en.wikipedia.org/wiki/Generalized_least_squares
- https://qiita.com/innovation1005/items/b712ce54a7a697a9bf03

上記のprint(arma43.mle_retvals)を見て，iterations の最大値（maxiter）は50とわかる。<br>
このことは次のソースコードを見て，maxiter=50 を確認できる。

- https://github.com/statsmodels/statsmodels/blob/main/statsmodels/tsa/arima/estimators/gls.py
    - maxiter=50


In [ ]:
arma54 = ARIMA(y, order=(5,0,4), trend='n').fit()

### 上記のsigma2, AICの結果のまとめ　&rArr;　本Notebookの最後「ARMA掲載(p,q)の同定結果の表示」に掲載
### ARMA(5,4)の極・零配置は，次の検証の中で掲載

## 真のシステムをARMA(2,1）のときの推定モデルの検証
ARMA(2,1)を用いる。他の次数はこの後半を参照されたい。

In [ ]:
print('arma21 -----------summary--------------------')
print(arma21.summary())

## 注意！
ARのパラメータの符号は反転している。これは，モデルの立て方でAR部の符号反転を行っているためである。  
http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_process.ArmaProcess.html  


このことは，次の例題でもARの符号を反転して与えていることからもわかる<br>
http://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_1.html  

また，極は$z$でなく$z^{-1}$で見ており，単位円外ならば安定極となっている。

#### plot_diagnosticsの使用
- https://ai-trend.jp/basic-study/time-series-analysis/sarima_model/
左上から順に、標準化した残差の時系列プロット、残差のヒストグラムと正規分布（とKDE分布）の密度関数、残差の正規QQプロット、残差の自己相関関数である。<br>
いずれも，ARMAモデルのパラメータ推定で，残差が白色性を示すとき，推定パラメータの不偏性が言える。このため，残差の時系列プロット以外の三つのグラフは，残差の白色性を検証していると言える。

In [ ]:
arma21.plot_diagnostics(figsize=(10,6))
#plt.savefig('fig_ARMAident_diagnostics.pdf', bbox_inches='tight')
plt.show()

#### 自己相関関数を用いた白色性の検定
上記の右下図は残差の自己相関関数$R(\tau)$を見ており，白色性ならば，$R(\tau)$は，$\tau=0$で1となるクロネッカーのデルタ関数となる。実際には，$\tau \ge 1$で$R(\tau)$はほぼ0が望ましいとする。これを検証する。<br>
自己相関関数を求める。これは，plot_diagnostics()の右下図と同じものを求める。<br>
alphaは信頼区間（1-alpha）を表す。

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
sm.graphics.tsa.plot_acf(arma21.resid, alpha=0.05, lags=10, ax=ax)
plt.show()

In [ ]:
resid = arma21.resid # residual
auto_corr = np.correlate(resid, resid, mode='full') #これは両側（0を中心として対称ゆえ，右半分だけ取り出す）
center = int(len(auto_corr)/2)
AutoR = auto_corr[center:]/np.max(auto_corr) #０を始点として右半分を抽出
print('Len(AutoR)=', len(AutoR))

fig = plt.subplots(figsize=(5,2))
plt.xlabel('k')
plt.plot(AutoR[:20], c='k', marker='o', linestyle='None')
plt.grid()

#plt.savefig('fig_ARMA_ParmEst_AutoCorr.pdf')

plt.show()

95\%（=(100 - 5)\%）信頼区間を求める。$R(\tau)$が漸近的に$N(0, 1/N)$に近づくことを利用して，次式の範囲を外れる$R(\tau)$の個数を調べる。
$$
| R(\tau)| \le \frac{1.96}{\sqrt{N}} 
$$
ここに，1.96は信頼区間95\%（両側で2.5\%ずつ）に対応するもので，次から求めた。

In [ ]:
sp.stats.norm.interval(confidence=0.95, loc=0.0, scale=1.0)

#### 上記の信頼区間を外れる自己相関関数の値の個数とその値を調べる。

In [ ]:
# 外れる個数のカウント　AutoRの全データ中，信頼区間を外れる個数は二桁となる。
print('No. of overing=',np.count_nonzero(np.abs(AutoR[1:]) > 1.96/np.sqrt(len(AutoR)) ))

In [ ]:
# ソートして，降順に並べることで，信頼区間を外れたデータの値を見る
np.sort(AutoR)[::-1][0:20]

## モデル評価
極，インパルス応答を見る。

In [ ]:
#　極配置を見る
poles_true = np.roots(ar)
ar_est = [1.0, -arma21.params[0], -arma21.params[1]]
ma_est = [1.0,  arma21.params[2]]
poles_est  = np.roots(ar_est)
print('True poles      =', poles_true)
print('Estimated poles =', poles_est)

fig, ax = plt.subplots(figsize=(3,3))
circ = patches.Circle(xy=(0, 0), radius=1.0, fill=False, ec='k')
ax.set_xlim((-1.2, 1.2))
ax.set_ylim((-1.2, 1.2))
ax.set_aspect('equal')
ax.add_patch(circ)

for i in range(2):
    ax.plot(np.real(poles_true[i]), np.imag(poles_true[i]), marker='+', color='r')
    ax.plot(np.real(poles_est[i]),  np.imag(poles_est[i]),  marker='x', color='b')
    
ax.set_xlabel('real part', fontsize=12)
ax.set_ylabel('imaginary part', fontsize=12)
ax.grid(linestyle='dashed', linewidth=1, c='gray')

#plt.savefig('fig_ARMAident_model_poles.pdf', bbox_inches='tight')

plt.show()

#### 上記のARMA(2,1)を真のシステムとしたとき，推定したARMA(5,4）モデルの極・零配置

In [ ]:
#ar54 = [1.0, -arma54.params[0], -arma54.params[1], -arma54.params[2], -arma54.params[3], -arma54.params[4]]
#ma54 = [1.0,  arma54.params[5],  arma54.params[6],  arma54.params[7],  arma54.params[8]]
ar54 = np.append(1.0, -arma54.params[0:5])
ma54 = np.append(1.0, arma54.params[5:9])
poles_est54  = np.roots(ar54)
zeros_est54   = np.roots(ma54)
print(poles_est54)
print(poles_true)
print('Estimated zeros=',zeros_est54)

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
circ = patches.Circle(xy=(0, 0), radius=1.0, fill=False, ec='k')
ax.set_xlim((-1.2, 1.2))
ax.set_ylim((-1.2, 1.2))
ax.set_aspect('equal')
ax.add_patch(circ)

# true system
for i in range(2):
    ax.plot(np.real(poles_true[i]), np.imag(poles_true[i]), marker='x', color='r')

ax.plot(-ma[1], 0.0, marker='o', color='r')
    

# ARMA(5,4) model
for i in range(5): 
    ax.plot(np.real(poles_est54[i]),  np.imag(poles_est54[i]),  marker='x', color='b')
    
for i in range(4):
    ax.plot(np.real(zeros_est54[i]),  np.imag(zeros_est54[i]),  marker='o', color='b')
    
    
ax.set_xlabel('real part', fontsize=12)
ax.set_ylabel('imaginary part', fontsize=12)
ax.grid(linestyle='dashed', linewidth=1, c='gray')

#plt.savefig('fig_ARMAident_model_poles_arma54.eps', bbox_inches='tight')

plt.show()

#### ARMA(2,1)のインパルス応答

In [ ]:
# インパルス応答を見る。
num=30
imp_res_true = arma_impulse_response(ar, ma, leads=num)
imp_res_est  = arma_impulse_response(ar_est, ma_est, leads=num)

fig, ax = plt.subplots( figsize=(6,3))
#　真のシステム
ax.plot(imp_res_true, c='k', linestyle='dashed', label='Actual system')
ax.grid()
ax.set_xlabel('k', fontsize=12)
ax.set_ylabel('y', fontsize=12)

#　推定したモデル
ax.plot(imp_res_est, c='k', label='Estimated model')
ax.grid()
ax.set_xlabel('k', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.legend()
ax.grid()

#plt.savefig('fig_ARMAident_model_impulse.pdf', bbox_inches='tight')
plt.show()

## 予測
推定したARMAパラメータを用いた予測を行う。ここでの予測の意味は，次のARMA(p,q)のモデル式より<br>
yt=ϕ1yt−1+…+ϕpyt−p+θ1ϵt−1+…+θqϵt−q+ϵt <br>
現時刻tのytの値を予測（システム制御論では推定）することになる。<br>
二つの予測方法を用いている。いずれも，安定したモデルと言える。

#### plot_predicを用いた予測
下記のstartからendの区間を予測する。途中からサンプル区間外となる。

In [ ]:
xstart, xend = N-100, N+50

fig, ax = plt.subplots(figsize=(7,3))
#fig, ax = plt.subplots(figsize=(14,3))

plot_predict(arma21, start=xstart, end=xend, ax=ax, dynamic=False)
# dynamic=Falseは，観測したサンプル値を予測に用いることを指定する。いずれ，モデル内で生成する入力がゼロに漸近するので，出力はゼロとなる。
# この期間はモデル次数とモデル特性に依存する。

ax.plot(y[xstart:], c='k', linestyle='dashed', label='y')

#ax.plot(np.arange(100,100+len(ytest)), ytest, label='real ytest')
x1, x2 = 100, 130
ax.plot(np.arange(x1,x2), ytest[:(x2-x1)], c='r', linestyle='dashed', label='ytest')
ax.legend(loc='upper left')

#plt.savefig('fig_ARMAident_model_predict.pdf', bbox_inches='tight')
plt.show()

#### get_prediction([start, end, dynamic, ...])を用いた予測
上記と同じことを行っている。

In [ ]:
pred = arma21.get_prediction(start=xstart, end=xend, dynamic=False)
pred_ci = pred.conf_int(alpha=0.05)

fig = plt.subplots(figsize=(6,3))
plt.plot(y[xstart:], c='k')
plt.plot(pred.predicted_mean, c='b', linestyle='dashed')
plt.fill_between(np.arange(len(pred_ci)), pred_ci[:, 0], pred_ci[:, 1], color='gray', alpha=0.2)

### メソッドforecastの使い方
メソッドforecastは，out of sampleの区間で用いられる。
この計算は，上記のget_prediction（）の結果に含まれる。このことは，下図の赤線を見ればわかる。

In [ ]:
nnn = 45
pred_extend = arma21.forecast(nnn)

fig = plt.subplots(figsize=(6,3))
plt.plot(y[xstart:], c='k')
plt.plot(pred.predicted_mean, c='b', linestyle='dashed')
n1 = N-xstart
n2 = n1 + nnn
#print(n1,n2)
#print(np.arange(n1,n2))
#len(pred_extend)
plt.plot(np.arange(n1,n2),pred_extend, c='r')

#### 残差の比較
残差の分散は次数(5,4)の方が小さいが，見た目では変わらない

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10,4))

ax[0].plot(y, label='y')
resid = arma21.resid  # short for residual
ax[0].plot(resid, label='resid')
ax[0].set_xlabel('k')
ax[0].legend()

ax[1].plot(y, label='y')
resid54 = arma54.resid  # short for residual
ax[1].plot(resid54, label='resid54')
ax[1].set_xlabel('k')
ax[1].legend()

#plt.savefig('fig_ARMA_ParmEst_PlotResid.pdf', bbox_inches='tight')
plt.show()

## 参考
参考までに自己相関関数を眼で見て周期を推定する述べる。下記に，
- 真の信号：sin関数()
- sig: 観測信号，真の信号に雑音が重畳している
下記のacf波形を見て，その周期を測る。

In [ ]:
num = 100
sig = np.sin(np.linspace(0.0, 6*np.pi, num)) + sp.stats.norm.rvs(loc=0.0, scale=0.1, size=num)
acf_np = np.correlate(sig, sig, mode='same')
acf_np = acf_np/acf_np.max()
plt.plot(sig, label='sig')
plt.plot(acf_np, label='acf')
plt.grid()
plt.legend()

## ARMA(p,q)の同定結果の表示

In [ ]:
print('arma20 -----------summary--------------------')
print(arma20.summary())

In [ ]:
print('arma21 -----------summary--------------------')
print(arma21.summary())

In [ ]:
print('arma32 -----------summary--------------------')
print(arma32.summary())

#### amra43の結果は収束計算に失敗しているが，参考までに表示する

In [ ]:
print('arma43 -----------summary--------------------')
print(arma43.summary())

In [ ]:
print('arma54 -----------summary--------------------')
print(arma54.summary())

#### 上記の結果を　sigma2, AICについて表形式にまとめる

In [ ]:
print('次数,   sigam2,     AIC')
print('  20   ', f'{arma20.params[2+0]:.4f}', '  ',f'{arma20.aic:.1f}')
print('  21   ', f'{arma21.params[2+1]:.4f}', '  ',f'{arma21.aic:.1f}')
print('  32   ', f'{arma32.params[3+2]:.4f}', '  ',f'{arma32.aic:.1f}')
print('  43   ', f'{arma43.params[4+3]:.4f}', '  ',f'{arma43.aic:.1f}')
print('  54   ', f'{arma54.params[5+4]:.4f}', '  ',f'{arma54.aic:.1f}')